## DIA 087: Visualizar Predicciones con Gráficas Usando Chart.js en Flask

Hoy vas a integrar gráficas interactivas a tu app Flask utilizando Chart.js, una librería de JavaScript que permite crear visualizaciones dinámicas y elegantes.
Vamos a graficar la cantidad de predicciones por clase para ver, por ejemplo, cuántas veces el modelo predijo "Clase 0" o "Clase 1".

✅ ¿Qué aprenderás hoy?

📌 Cómo integrar Chart.js en una app Flask.

📌 Cómo enviar datos desde Flask a la plantilla HTML.

📌 Cómo graficar datos dinámicos desde la base de datos SQLite.

🗂️ Estructura del Proyecto
pgsql
Copiar
Editar
titanic_app/
├── app.py
├── model.pkl
├── db.sqlite3
└── templates/
    ├── index.html
    ├── historial.html
    └── grafica.html ✅ NUEVA
🧩 1. Código Flask Actualizado (app.py)
Agregamos una nueva ruta /grafica para enviar datos al frontend:

python
Copiar
Editar
from flask import Flask, render_template, request, send_file
from flask_sqlalchemy import SQLAlchemy
import joblib
import numpy as np
import datetime
import pandas as pd
import io
from collections import Counter

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
db = SQLAlchemy(app)

class Registro(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    f1 = db.Column(db.Float)
    f2 = db.Column(db.Float)
    f3 = db.Column(db.Float)
    f4 = db.Column(db.Float)
    prediccion = db.Column(db.String(50))
    fecha = db.Column(db.DateTime, default=datetime.datetime.utcnow)

modelo = joblib.load("model.pkl")

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predecir", methods=["POST"])
def predecir():
    try:
        f1 = float(request.form["f1"])
        f2 = float(request.form["f2"])
        f3 = float(request.form["f3"])
        f4 = float(request.form["f4"])

        datos = np.array([[f1, f2, f3, f4]])
        pred = modelo.predict(datos)[0]

        nuevo_registro = Registro(f1=f1, f2=f2, f3=f3, f4=f4, prediccion=str(pred))
        db.session.add(nuevo_registro)
        db.session.commit()

        return render_template("index.html", resultado=f"✅ Resultado: Clase {pred}")
    except:
        return render_template("index.html", resultado="❌ Error en los datos")

@app.route("/grafica")
def grafica():
    registros = Registro.query.all()
    predicciones = [r.prediccion for r in registros]
    conteo = Counter(predicciones)
    
    clases = list(conteo.keys())
    cantidades = list(conteo.values())

    return render_template("grafica.html", clases=clases, cantidades=cantidades)

if __name__ == "__main__":
    db.create_all()
    app.run(debug=True)
🖼️ 2. Plantilla HTML (templates/grafica.html) con Chart.js
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Gráfica de Predicciones</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
<div class="container mt-5">
    <h2 class="text-center">📈 Gráfica de Predicciones por Clase</h2>
    <a href="/" class="btn btn-primary mb-3">← Volver</a>

    <canvas id="graficaPredicciones" width="400" height="200"></canvas>
</div>

<script>
    const ctx = document.getElementById('graficaPredicciones').getContext('2d');
    const grafico = new Chart(ctx, {
        type: 'bar',
        data: {
            labels: {{ clases|tojson }},
            datasets: [{
                label: 'Cantidad de Predicciones',
                data: {{ cantidades|tojson }},
                backgroundColor: 'rgba(54, 162, 235, 0.7)',
                borderColor: 'rgba(54, 162, 235, 1)',
                borderWidth: 1
            }]
        },
        options: {
            scales: {
                y: { beginAtZero: true }
            }
        }
    });
</script>
</body>
</html>
🔍 Líneas Clave Explicadas
Counter(predicciones): cuenta cuántas veces aparece cada clase.

{{ clases|tojson }}: pasa la lista de clases al frontend como JSON.

Chart.js: crea una barra por clase con su cantidad respectiva.

type: 'bar': tipo de gráfico (puedes cambiar a 'pie', 'line', etc.).